In [1]:
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from sklearn.metrics import accuracy_score

In [3]:
client = OpenAI()

In [4]:
assistant = client.beta.assistants.create(
  name="PDF searcher",
  instructions="Мне необходимо, чтобы ты в pdf файлах искал ответы на заданный вопрос. Правильный овтет лишь один",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [6]:
path = "practice7_data/practice7_data"

items = os.listdir(path)

files = [os.path.join(path, f) for f in items]

vector_store = client.beta.vector_stores.create(name="Data for searching")

file_streams = [open(path, "rb") for path in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(vector_store_id=vector_store.id, files=file_streams)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [7]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

NameError: name 'assistant' is not defined

In [38]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Термин первая помощь"
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    
full_ans = messages[0].content[0].text


annotations = full_ans.annotations
citations = []
for index, annotation in enumerate(annotations):
    full_ans.value = full_ans.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))